In [128]:
import pandas as pd
import pickle
import os
import nltk
#nltk.download('wordnet')
import random
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from tqdm import tqdm_notebook as tqdm

In [121]:
#All excel file paths here, pickle paths are used by other scripts so hardcoded
master_file = "./data/master_df_labeled.xlsx"
#new sampled data to be labeled
new_file = "./data/please_label.xlsx"
grounds_file = "./data/ground_truths.xlsx"
batch_folder = "./data/please_label_batch/100/"

In [138]:
#Generates new sample of data to be labeled
def please_label(master_file, new_file, sample_num=100):
  master_df = pd.read_excel(master_file,index_col=0)
  to_label_df = master_df[master_df.true_pos==-1]
  output_df = to_label_df.sample(sample_num)
  output_df.to_excel(new_file)
  return output_df

#Create fixed chunks of shuffled data for people to label
def batch_please_label(master_file, batch_folder, sample_num=100):
  master_df = pd.read_excel(master_file,index_col=0).drop(labels=["lemmatized","problematic"],axis=1)
  to_label_df = master_df[master_df.true_pos==-1].sample(frac=1,random_state=0)
  row_num = to_label_df.shape[0]
  batches = row_num//sample_num
  print(batches)
  #assert 1==0
  
  for start_bin in range(batches):
    end_bin = start_bin+1
    temp = to_label_df.iloc[start_bin*sample_num:end_bin*sample_num,:]
    if end_bin == batches and row_num%sample_num>0:#we have leftover rows to deal with
      extra = to_label_df.iloc[end_bin*sample_num:,:]
      extra.to_excel(batch_folder+"batch_"+str(end_bin)+".xlsx")
    temp.to_excel(batch_folder+"batch_"+str(start_bin)+".xlsx")
  print(f"Full Batches: {batches}")
  print(f"Partial Batch?: {row_num%sample_num>0}")
  print(f"Go check {batch_folder}")
    
  
#Update our running master_df_labeled with multiple new files
def batch_merge_into_master(master_file, batch_folder, grounds_file):
  batch_files = os.listdir(batch_folder)
  for new_file in tqdm(batch_files):
    master_df_labeled, ground_truths_df = merge_into_master(master_file, new_file, grounds_file, batch=True)
  ground_truths_df = produce_ground_truths(master_df_labeled, grounds_file)
  master_df_labeled.to_excel(master_file)
  pickle.dump(master_df_labeled,open("./data/master_df_labeled.p","wb"))
  return master_df_labeled, ground_truths_df

#Update our running master_df_labeled, main function to produce files for dataloaders
def merge_into_master(master_file, new_file, grounds_file, batch=False):
  master_df_labeled = pd.read_excel(master_file,index_col=0)
  new_df = pd.read_excel(new_file,index_col=0)
  master_df_labeled.update(new_df.true_pos)
  if not batch:
    ground_truths_df = produce_ground_truths(master_df_labeled, grounds_file)
    master_df_labeled.to_excel(master_file)
    pickle.dump(master_df_labeled,open("./data/master_df_labeled.p","wb"))
  return master_df_labeled, ground_truths_df

#Produces ground_truths_df
def produce_ground_truths(master_df_labeled, grounds_file):
  ground_truths_df = master_df_labeled[master_df_labeled.true_pos!=-1]
  ground_truths_df.to_excel(grounds_file)
  pickle.dump(ground_truths_df,open("./data/ground_truths_df.p","wb"))
  return ground_truths_df

#Print out how many examples are labeled as True or False Positives
def print_tf_pos(master_df_labeled):
  print(f"Count of true positives: {master_df_labeled[master_df_labeled.true_pos==1].shape[0]}")
  print(f"Count of false positives: {master_df_labeled[master_df_labeled.true_pos==0].shape[0]}")
  assert master_df_labeled[master_df_labeled.problematic==0].shape[0]==0

# Example Workflow
1. Print current true and false positive counts 
2. Create new file of reviews to be labeled (please_label)
3. After labeling, update our running master file (merge_into_master)
4. Print new true and false positive counts to check, also select specific example to check

In [95]:
#Print current true and false positive counts
print_tf_pos(master_df_labeled)

Count of true positives: 52
Count of false positives: 44


In [139]:
#Generate fixed batch to label
batch_please_label(master_file, batch_folder, sample_num=100)

1109
Full Batches: 1109
Partial Batch?: True
Go check ./data/please_label_batch/100/


In [109]:
#Update running master_df_labeled and ground_truths_df,
#Print new counts and an example to check
master_df_labeled , ground_truths_df = merge_into_master(master_file, new_file, grounds_file)
print_tf_pos(master_df_labeled)
master_df_labeled.loc[98209,:]

Count of true positives: 52
Count of false positives: 47


review           ['did', 'a', 'great', 'job', 'on', 'our', 'tea...
flagged_word                                                 great
flagged_index                                                    2
problematic                                                      1
lemmatized                                                   great
true_pos                                                         0
Name: 98209, dtype: object

# Scratch

In [142]:
# checklist_dict = {"file":["batch_"+str(x) for x in range(1111)], "sent_to":['' for x in range(1111)], "completed":['' for x in range(1111)]}
# master_checklist = pd.DataFrame(checklist_dict).to_excel("./data/please_label_batch/master_checklist.xlsx")

In [114]:
test = master_df_labeled[master_df_labeled.true_pos==-1]
test.shape

(110930, 6)

In [117]:
shuffled = test.sample(frac=1)
shuffled.head(20)

,review,flagged_word,flagged_index,problematic,lemmatized,true_pos
55604,"['it', 'is', 'a', 'genuine', 'privilege', 'to'...",winner,14,1,winner,-1.0
39918,"['superb', 'designers', '.']",superb,0,1,superb,-1.0
65186,"['lepro', ',', 'was', 'great', 'easy', 'to', '...",great,3,1,great,-1.0
102737,"['very', 'reliable', 'and', 'easy', 'to', 'wor...",reliable,1,1,reliable,-1.0
71007,"['responsive', 'and', 'a', 'great', 'communica...",great,3,1,great,-1.0
126709,"['she', 'was', 'great', ',', 'quick', 'respons...",understanding,9,1,understanding,-1.0
111335,"['thank', 'you', 'for', 'your', 'great', 'work...",great,4,1,great,-1.0
44386,"['hayley', 'is', 'an', 'excellent', 'provider'...",excellent,3,1,excellent,-1.0
87630,"['he', 'is', 'great', 'and', 'fast', 'and', 'r...",great,2,1,great,-1.0
111267,"['he', 'was', 'very', 'eager', 'to', 'help', '...",polite,8,1,polite,-1.0


In [122]:
shuffled.iloc[5:10,:]

,review,flagged_word,flagged_index,problematic,lemmatized,true_pos
126709,"['she', 'was', 'great', ',', 'quick', 'respons...",understanding,9,1,understanding,-1.0
111335,"['thank', 'you', 'for', 'your', 'great', 'work...",great,4,1,great,-1.0
44386,"['hayley', 'is', 'an', 'excellent', 'provider'...",excellent,3,1,excellent,-1.0
87630,"['he', 'is', 'great', 'and', 'fast', 'and', 'r...",great,2,1,great,-1.0
111267,"['he', 'was', 'very', 'eager', 'to', 'help', '...",polite,8,1,polite,-1.0


# Initial Labeling Process

In [81]:
master_df = pickle.load(open("./data/master_df.p", "rb"))
master_df["lemmatized"] = [wordnet_lemmatizer.lemmatize(w) for w in master_df.flagged_word]
#
print(len(set(master_df.flagged_word)))
print(len(set(master_df.lemmatized)))
#print(set(master_df.flagged_word)-set(master_df.lemmatized))
# print(sorted(list(set(master_df.lemmatized)),key=str.lower))
# master_df.shape

524
448


In [82]:
#Get counts of reviews for each word
grouped = master_df.groupby(master_df.lemmatized)['review']
counts = pd.DataFrame(grouped.count())


In [75]:
#Store into excel file
counts.sort_values('review',ascending=False).to_excel("./data/counts.xlsx")

In [8]:
# #creating random sample from master_df to label
# sampled = master_df[master_df.problematic==1].sample(500)
# sampled.to_excel("./data/fptp.xlsx")

In [76]:
#merge by row index
labeled = pd.read_excel("./data/fptp_labeled.xlsx",index_col=0)
master_df_labeled = master_df.join(labeled.true_pos).fillna(-1)
master_df_labeled = master_df_labeled[master_df_labeled.problematic==1]


In [11]:
ground_truths_df = master_df_labeled[master_df_labeled.true_pos!=-1]
pickle.dump(ground_truths_df,open("./data/ground_truths_df.p","wb"))

In [15]:
ground_truths_df

,review,flagged_word,flagged_index,problematic,true_pos
9133,"[great, job, as, usual-above, and, beyond, cal...",great,0,1,0.0
14637,"[he, is, intelligent, ,, dedicated, ,, passion...",intelligent,2,1,1.0
15243,"[great, work, hire, him, again]",great,0,1,0.0
15541,"[strong, ability, to, execute, feedback, with,...",great,6,1,0.0
16159,"[awesome, +, dependable, designer, !]",dependable,2,1,1.0
...,...,...,...,...,...
135248,"[very, understanding, and, flexible, with, som...",understanding,1,1,1.0
136587,"[the, designer, is, very, intuitive, .]",intuitive,4,1,1.0
137606,"[great, designs, .]",great,0,1,0.0
138055,"[incredibly, detailed, oriented, ,, efficient,...",efficient,4,1,1.0


In [84]:
#master file to continuously update
master_df_labeled.to_excel("./data/master_df_labeled.xlsx")
pickle.dump(master_df_labeled,open("./data/master_df_labeled.p","wb"))